<img src="Assets/feup.png" alt="Drawing" style="float : left; width: 200px;"/>

Temática 4: Aprendizagem Supervisionada - Regressão
==================================================
## Coronavirus Disease (COVID-19) Situation Reports

### Entrega final

### Inteligência Artificial(IART) - 3º ano Mestrado em Engenharia Informática e Computação

#### Autores:
&nbsp; Cláudia Martins - up201704136

&nbsp; Andreia Gouveia - up201706430

&nbsp; Filipa Senra    - up201704077

## Introdução

O presente trabalho tem como finalidade exporar os Modelos de Regressão da Aprendizagem Supervisonada. O tema que nos foi Proposto é a Temática sobre a **Situação do Covid-19** em várias Regiões. Deste modo, pretendemos prever o número de Novos Casos e Mortes num dado Dia (após o 1º caso ter sido reportado na região). 
O relatório está organizado em 5 partes. Na Parte 1, será feita um Descrição do Problema, descrevendo os *Data Sets* utilizados para a aplicação dos vários algoritmos. Na Parte 2, será abordado a Resolução da Problemática, sendo apresentados os passos realizados para a solução do problema. Na Parte 3, serão avaliados os Resultados Obtidos pela Solução Encontrada, através de Gráficos mostrando tanto os resultados obtidos, bem como o erro associado a estes. Na Parte 4, serão apresentadas as Conclusões elencadas através do Resultados Experimentais Obtidos.

## Descrição do Problema

Foram utilizados neste trabalho dois *Data Sets* que podem ser encontrados [aqui](https://www.kaggle.com/imdevskp/corona-virus-report) e [aqui](https://www.kaggle.com/fernandol/countries-of-the-world).

O primeiro *Data Set* é composto pelas informações sobre o desenvolvimento dos casos de Covid-19.
Cada linha é composta por Província/Estado, País/Região, Latitude, Longitude, Data, Casos Confirmados, Mortes e Recuperados. Optamos por não usar as colunas da Latitude e Longitude e agrupar o número de Casos Confirmados, Mortes e Recuperados por País. Foi, de igual modo, criada uma nova coluna que representa o número de dias desde que o 1º caso ocorreu no País/Região. Deste modo, a coluna Data, também, não foi utilizada. Foi, também, adicionado o Número de Novos Casos, Mortos e Recuperados do dia anterior.

O segundo *Data Set* é composto pelas informações demográficas de cada País. Optamos por usar as colunas: Taxa de Mortalidade, Taxa de Mortalidade Infantil, Produto Interno Bruto *per capita*, Número de Habitantes e a Densidade Populacional. Escolhemos incluir a Taxa de Mortalidade Infantil e a Taxa Mortalidade, pois são reconhecidas como boas métricas sobre o Estado de Desenvolvimento de um País bem como a qualidade da Saúde no País. O Produto Interno Bruto *per capita* foi utilizado, pois países mais ricos poderão encarar a doença com mais meios. A Densidade Populational foi utilizada, pois a transmidação do Covid-19 é maior em zonas mais densamente populacionadas. A Número de Habitantes foi utilizado, pois possui uma forte corrulação entre o número de casos de Covid-19.

A partir destes dois *Data Sets* foi construido um final que agrupa todas as informações. Cada linha é composta por: País/Região, Número de Habitantes, Densidade Populacional, Produto Interno Bruto *per capita*, Taxa de Mortalidade, Taxa de Mortalidade Infantil, Número de Dias desde o 1º Caso Reportado na Região, Número de Novos Casos Confirmados, Número de Novas Mortes Confirmadas, Número de Novos Recuperados, Número de Novos Casos do Dia Anterior, Número de Mortos do Dia Anterior e Número de Recuperados do Dia Anterior.

## Abordagem

Tanto os *Data Sets* de treino como de Teste foram extraídos *Data Set* obtido anteiormente. Esta extração foi feita de forma aleatória, recorrendo à função *train_test_split* com os pârametros *test_size* a 0.2 e o *random_state* a 0. Assim,

### Construção do *Data Set* a ser utilizado

#### Depências e Bibliotecas a ser utilizadas

In [1]:
!pip install pandas
!pip install matplotlib
!pip install seaborn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import preprocessing as pre
from sklearn.datasets import make_regression
from sklearn import neighbors
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPRegressor

import matplotlib.pyplot as plt
import seaborn as sb

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


#### 1. Importação do *Data Set* com os Dados Relativos ao Covid-19

In [2]:
covid_data = pd.read_csv('covid_19_clean_complete.csv')

In [3]:
covid_data.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,33.0000,65.0000,1/22/20,0,0,0
1,NaN,Albania,41.1533,20.1683,1/22/20,0,0,0
2,NaN,Algeria,28.0339,1.6596,1/22/20,0,0,0
3,NaN,Andorra,42.5063,1.5218,1/22/20,0,0,0
4,NaN,Angola,-11.2027,17.8739,1/22/20,0,0,0


#### 1.1 Eliminação das Colunas que não irão ser utilizadas e Passagem da Coluna "Data" para Objeto Data de *Python* 

In [4]:
covid_data.drop(['Province/State', 'Lat', 'Long'],axis=1, inplace=True)
covid_data.head()

,Country/Region,Date,Confirmed,Deaths,Recovered
0,Afghanistan,1/22/20,0,0,0
1,Albania,1/22/20,0,0,0
2,Algeria,1/22/20,0,0,0
3,Andorra,1/22/20,0,0,0
4,Angola,1/22/20,0,0,0


#### 1.2 Agregação dos Casos por País/Região, Eliminação das Colunas com Data anterior ao 1º Caso Reportado no País/Região

In [5]:
#aggregates de countries by date
covid_data = covid_data.groupby(['Country/Region', 'Date'])[['Confirmed', 'Deaths', 'Recovered']].apply(sum)

#eliminates rows with no suspected, no deaths and no cured
covid_data = covid_data[(covid_data.T != 0).any()] 

#get group by columns back
covid_data.reset_index(inplace=True) 

covid_data.head(10)

,Country/Region,Date,Confirmed,Deaths,Recovered
0,Afghanistan,2/24/20,1,0,0
1,Afghanistan,2/25/20,1,0,0
2,Afghanistan,2/26/20,1,0,0
3,Afghanistan,2/27/20,1,0,0
4,Afghanistan,2/28/20,1,0,0
5,Afghanistan,2/29/20,1,0,0
6,Afghanistan,3/1/20,1,0,0
7,Afghanistan,3/10/20,5,0,0
8,Afghanistan,3/11/20,7,0,0
9,Afghanistan,3/12/20,7,0,0


#### 1.3 Uniformalização dos Nomes dos Países e Eliminação de Regiões/Países Não Válidos

In [6]:
#get countries present in covid data and removing trailing white spaces
covid_data_country_unique = covid_data['Country/Region'].drop_duplicates().str.strip()
pd.set_option('display.max_rows', covid_data_country_unique.shape[0]+1)
print(covid_data_country_unique)

#deleting rows with missing data
covid_data.dropna(inplace=True)

0                             Afghanistan
74                                Albania
134                               Algeria
207                               Andorra
274                                Angola
323                   Antigua and Barbuda
379                             Argentina
445                               Armenia
513                             Australia
616                               Austria
689                            Azerbaijan
757                               Bahamas
810                               Bahrain
884                            Bangladesh
945                              Barbados
997                               Belarus
1067                              Belgium
1161                               Belize
1207                                Benin
1260                               Bhutan
1323                              Bolivia
1381               Bosnia and Herzegovina
1445                             Botswana
1484                              

#### 2. Importação do *Data Set* com os Dados Demográficos dos Países

In [7]:
countries_data = pd.read_csv('countries_data.csv', decimal=',')

#### 2.1 Eliminação das Colunas que não irão ser utilizadas e de Linhas com Valores em Falta

In [8]:
#deleting useless columns from countries data
countries_data.drop(['Region', 'Net migration', 'Area (sq. mi.)','Coastline (coast/area ratio)', 'Literacy (%)', 'Phones (per 1000)', 'Arable (%)','Crops (%)','Other (%)','Climate','Birthrate','Agriculture','Industry','Service'],axis=1, inplace=True)

#deleting countries with missing data
countries_data.dropna(inplace=True)

#### 2.2 Visualização das Diferenças de Países entre os "DataSets*

In [9]:
#removing trailing white spaces
countries_data_country = countries_data['Country'].str.strip()

#replacing & for and
countries_data_country.replace({'&': 'and'}, inplace=True, regex=True)

#get series with all the countries present in both data sets
df = pd.concat([covid_data_country_unique,countries_data_country])

#drop values that are repeated (keep the unique values)
unique_values = df.drop_duplicates(keep=False)

#sort them for an easy read
unique_values = unique_values.sort_values()

pd.set_option('display.max_rows', unique_values.shape[0]+1)
print(unique_values)

3                  American Samoa
6                        Anguilla
10                          Aruba
757                       Bahamas
14                   Bahamas, The
22                        Bermuda
28             British Virgin Is.
1816                   Cabo Verde
37                     Cape Verde
38                 Cayman Islands
39           Central African Rep.
2133     Central African Republic
2481          Congo (Brazzaville)
2535             Congo (Kinshasa)
45               Congo, Dem. Rep.
46           Congo, Repub. of the
53                 Czech Republic
2904                      Czechia
3043             Diamond Princess
58                     East Timor
3675                     Eswatini
66                  Faroe Islands
70                  French Guiana
71               French Polynesia
4096                       Gambia
73                    Gambia, The
74                     Gaza Strip
78                      Gibraltar
80                      Greenland
82            

#### 2.3 Uniformazição dos Nomes dos Países nos Dois *Data Sets*

In [10]:
#particular cases
countries_data_country.replace({'Taiwan': 'Taiwan*'}, inplace=True, regex=True)
countries_data_country.replace({'Macedonia': 'North Macedonia'}, inplace=True, regex=True)
countries_data_country.replace({'Korea, South': 'South Korea'}, inplace=True, regex=True)
countries_data_country.replace({'United States': 'US'}, inplace=True, regex=True)
countries_data_country.replace({'Gambia, The': 'Gambia'}, inplace=True, regex=True)
countries_data_country.replace({'American Samoa': 'Samoa'}, inplace=True, regex=True)
countries_data_country.replace({'Czech Republic': 'Czechia'}, inplace=True, regex=True)
countries_data_country.replace({'Swaziland': 'Eswatini'}, inplace=True, regex=True)
countries_data_country.replace({'East Timor': 'Timor-Leste'}, inplace=True, regex=True)
countries_data_country.replace({'Bahamas, The': 'Bahamas'}, inplace=True, regex=True)
countries_data_country.replace({'Cape Verde': 'Cabo Verde'}, inplace=True, regex=True)
countries_data_country.replace({'Central African Rep.': 'Central African Republic'}, inplace=True, regex=True)
countries_data_country.replace({'Congo, Dem. Rep.': 'Congo (Kinshasa)'}, inplace=True, regex=True)
countries_data_country.replace({'Congo, Repub. of the': 'Congo (Brazzaville)'}, inplace=True, regex=True)

#replacing the column with the country names with the revised column
countries_data_df = pd.DataFrame(countries_data)
countries_data_country_df = pd.DataFrame(countries_data_country)
new_countries_data_df = countries_data_df.assign(Country=countries_data_country_df['Country'])

#### 2.4 Eliminação dos Países/Regiões sem Correspondência nos Dois *Data Sets*

In [11]:
#eliminating countries from the countries_data that are not in the covid_data
countries_data = new_countries_data_df

for x in unique_values:
  countries_data.drop(countries_data[countries_data['Country'] == x ].index , inplace=True)
  covid_data.drop(covid_data[covid_data['Country/Region'] == x ].index , inplace=True)

#### 2.5 Exportação dos *Data Sets* indivíduais limpos

In [12]:
#Saving cleaned data_set into to a file
countries_data.to_csv('countries_data_clean.csv')
covid_data.to_csv('covid_data_clean.csv')

#### 3. Junção dos Dois *Data Sets*

#### 3.1 Junção dos Países em Linhas com todas as Informações

In [13]:
#Merge the two columns together and drop uncessary columns
merged_inner = pd.merge(left=covid_data, right=countries_data, left_on='Country/Region', right_on='Country').drop(columns=['Country'])

#Drop rows with at least one missing value
merged_inner.dropna(inplace=True)

merged_inner.head(50)

,Country/Region,Date,Confirmed,Deaths,Recovered,Population,Pop. Density (per sq. mi.),Infant mortality (per 1000 births),GDP ($ per capita),Deathrate
0,Afghanistan,2/24/20,1,0,0,31056997,48.0,163.07,700.0,20.34
1,Afghanistan,2/25/20,1,0,0,31056997,48.0,163.07,700.0,20.34
2,Afghanistan,2/26/20,1,0,0,31056997,48.0,163.07,700.0,20.34
3,Afghanistan,2/27/20,1,0,0,31056997,48.0,163.07,700.0,20.34
4,Afghanistan,2/28/20,1,0,0,31056997,48.0,163.07,700.0,20.34
5,Afghanistan,2/29/20,1,0,0,31056997,48.0,163.07,700.0,20.34
6,Afghanistan,3/1/20,1,0,0,31056997,48.0,163.07,700.0,20.34
7,Afghanistan,3/10/20,5,0,0,31056997,48.0,163.07,700.0,20.34
8,Afghanistan,3/11/20,7,0,0,31056997,48.0,163.07,700.0,20.34
9,Afghanistan,3/12/20,7,0,0,31056997,48.0,163.07,700.0,20.34


#### 3.2 Criação de Nova Coluna que Representa os Dias Decorridos desde o 1º Caso Reportado, Criação de Colunas com os Dados do Dia Anterior

In [14]:
#get group by columns back
merged_inner.reset_index(inplace=True) 
merged_inner['Date'] = pd.to_datetime(merged_inner['Date'])

#substructs the mim date from the current date (and returns the result in days - dt.days)
merged_inner['Ellapsed Days'] = (merged_inner['Date'] - merged_inner.groupby('Country/Region')['Date'].transform('min')).dt.days


def createPrevValues(dataset, originalColumnName, commonColumnName, newColumnName):
    dataset[newColumnName] = dataset.loc[dataset[commonColumnName].shift(-1)==dataset[commonColumnName], originalColumnName]
    dataset[newColumnName] = dataset[newColumnName].shift()
    dataset[newColumnName].fillna(0, inplace=True)

#Get cases of previous day
createPrevValues(merged_inner, 'Confirmed', 'Country/Region', 'Prev_Confirmed')

#Get deaths of previous day
createPrevValues(merged_inner, 'Deaths', 'Country/Region', 'Prev_Deaths')

#Get Recovered of previous day
createPrevValues(merged_inner, 'Recovered', 'Country/Region', 'Prev_Recovered')

merged_inner.drop(columns=['Date'], inplace=True)

merged_inner.head()

,index,Country/Region,Confirmed,Deaths,Recovered,Population,Pop. Density (per sq. mi.),Infant mortality (per 1000 births),GDP ($ per capita),Deathrate,Ellapsed Days,Prev_Confirmed,Prev_Deaths,Prev_Recovered
0,0,Afghanistan,1,0,0,31056997,48.0,163.07,700.0,20.34,0,0.0,0.0,0.0
1,1,Afghanistan,1,0,0,31056997,48.0,163.07,700.0,20.34,1,1.0,0.0,0.0
2,2,Afghanistan,1,0,0,31056997,48.0,163.07,700.0,20.34,2,1.0,0.0,0.0
3,3,Afghanistan,1,0,0,31056997,48.0,163.07,700.0,20.34,3,1.0,0.0,0.0
4,4,Afghanistan,1,0,0,31056997,48.0,163.07,700.0,20.34,4,1.0,0.0,0.0


# TODO: check with graphs the outliers

#### 3.3 Exportação do *Data Set* final

In [15]:
merged_inner.to_csv('covid_and_country_data.csv')

### Obtenção dos Valores que os Modelos irão receber

In [16]:
#Get set with the values that the model receives
X = {'Confirmed' : merged_inner.drop(columns=[ 'Country/Region', 'Confirmed', 'Recovered']) , 'Deaths' : merged_inner.drop(columns=[ 'Country/Region', 'Deaths', 'Recovered'])}

#Values the model should predict
y = {'Confirmed' : merged_inner['Confirmed']  , 'Deaths' : merged_inner['Deaths']}

### Decision Tree Regression

In [17]:
#Fitting model with tranning set
variables = ['Confirmed' , 'Deaths']

for variable in variables:
    
    print('On var ' , variable)
    
    #Setting up training sets
    X_train, X_test, y_train, y_test = train_test_split(X[variable], y[variable], test_size=0.2, random_state=0)
    regressor = DecisionTreeRegressor(random_state = 0)
    regressor.fit(X_train, y_train)
    
    #Predicting
    y_pred = regressor.predict(X_test)
    
    #Show difference of real and predicted values
    df=pd.DataFrame({'Actual':y_test, 'Predicted':y_pred})
    df

    #Show Errors
    print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
    print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
    print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

On var  Confirmed


NameError: name 'DecisionTreeRegressor' is not defined

### Nearest Neighbors Regression

In [ ]:
variables = ['Confirmed' , 'Deaths']
num_nei = [1 , 2 , 3 , 5 , 10 , 15 , 20 , 25]

for variable in variables:
    print("On " , variable)
    X_train, X_test, y_train, y_test = train_test_split(X[variable], y[variable], test_size=0.2, random_state=0)
    
    for num in num_nei:
        knn = neighbors.KNeighborsRegressor(num , weights='distance')

        knn.fit(X_train , y_train)

        y_pred = knn.predict(X_test)

        df=pd.DataFrame({'Actual':y_test, 'Predicted':y_pred})
        print(df)

        #Show Errors
        print("---------ERRORS-----------")
        print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
        print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
        print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred))) 
        print('--------')
        
print('Done')        

### Neural Network Regression

In [ ]:
#Test case
variables = ['Confirmed' , 'Deaths']
    
#different nr of iterations to compare results
max_iter_array = [150, 200, 250 , 300 , 350]


for variable in variables:
    #Setting up training sets
    
    X_train, X_test, y_train, y_test = train_test_split(X[variable], y[variable], test_size=0.2, random_state=0)
    print('On variable : ' , variable)
    
    for it in max_iter_array:
        print('On iteration ', it)
        mlpr = MLPRegressor(solver = "lbfgs" , max_iter=it)

        #need to fix date
        mlpr.fit(X_train , y_train)

        #predict model
        #predict_train = mlpr.predict(X_train_scaled)

        #predict test
        y_pred = mlpr.predict(X_test)
        print('Predict: ' , y_pred)

        df=pd.DataFrame({'Actual':y_test, 'Predicted':y_pred})
        print(df)

        #Show Errors
        print("---------ERRORS-----------")
        print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
        print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
        print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred))) 
        print('--------')
    
      
print("Done")

In [ ]:
#Values to compare with error -> less than 10% of the mean result is a good success
merged_inner.describe()